## Ver cómo puedo cruzar por fecha aproximada

In [1]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "12g")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("ReadParquetYearMon16MM").config(conf=conf).getOrCreate()
spark

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/17 17:57:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
import time
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

Inicio: Mon Oct 17 17:57:44 2022


In [ ]:
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
from pyspark.sql.functions import sum, col, desc

esquema = StructType() \
      .add("Id",StringType(),True) \
      .add("fecNac",StringType(),True) \
      .add("sex",StringType(),True) \
      .add("ingresos",IntegerType(),True) \
      .add("partido",StringType(),True) \
      .add("edad",IntegerType(),True) \
      .add("mapa",StringType(),True)

"""
 * Lectura de un rango de meses de un parquet particionado por año/mes:
"""

def listPartitions (anoMesFinal, meses, path):
    lista = []
    mes = int(anoMesFinal[-2:]) + 1
    ano = int(anoMesFinal[:-3])
    
    print('Primer mes: '+anoMesFinal)
    
    for i in range(meses):
        mes = mes - 1
        if mes == 0:
            ano = ano - 1
            mes = 12
        if mes < 10: anoMes = str(ano)+'-0'+str(mes)
        else: anoMes = str(ano)+'-'+str(mes)
        #print('Desde la función: '+str(i+1)+' - '+anoMes)

        lista.append(path+anoMes)
    print('Último mes: '+anoMes)
    return lista

df = spark.read.option("mergeSchema", True).option("schema", "esquema").option("basePath", "file:///opt/workspace/datos/fecNac/fakePeople800M.parquet/")\
               .format("parquet").load(listPartitions('2004-06', 1, './datos/fecNac/fakePeople800M.parquet/yearMon='))
                                                                                                   
df.printSchema()

print(df.count())
df.show()
df.explain(extended=True)
df.explain(mode='cost')


Primer mes: 2004-06
Último mes: 2004-06


root
 |-- Id: string (nullable = true)
 |-- fecNac: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- ingresos: integer (nullable = true)
 |-- partido: string (nullable = true)
 |-- edad: integer (nullable = true)
 |-- mapa: string (nullable = true)
 |-- yearMon: string (nullable = true)



In [ ]:
print ('Final: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicio, 4)} segundo(s)')

In [ ]:
#df.groupBy('yearMon').count().sort(desc('yearMon')).show(200)

In [ ]:
esqPartF = StructType() \
      .add("partido",StringType(),True) \
      .add("nombre",StringType(),True) \
      .add("lider",StringType(),True) \
      .add("fechaInicio",StringType(),True) \
      .add("fechaFin",StringType(),True)


dfPartF = spark.read.csv("./datos/partidosFechas.csv", sep=";", header = True, schema = esqPartF)

dfPartF.show()

In [ ]:
dfJ = df.join(dfPartF, (df.partido == dfPartF.partido) & (df.fecNac >= dfPartF.fechaInicio) & (df.fecNac <= dfPartF.fechaFin), "inner").drop(dfPartF.partido)
dfJ.show()
dfJ.count()

In [ ]:
dfJ.filter('partido ="VOX"').show(200)

In [ ]:
dfCruzan = dfJ.select(col("Id"))
dfCruzan.show()

In [ ]:
# Probando el left_anti. Nada que ver con la celda anterior
dfKK = df.join(dfJ, df.Id == dfJ.Id, "left_anti")
dfKK.show()

In [ ]:
dfKK.groupBy('partido').count().show()

In [ ]:
spark.sparkContext.stop()
print('Sacabao')